<a href="https://colab.research.google.com/github/kridtapon/Retrace-Cross-Strategy/blob/main/Retrace_%26_Cross_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.4/527.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninsta

In [54]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

def calculate_fibonacci_levels(df, lookback=50, fib_levels=None):
    """Calculate Fibonacci retracement levels based on recent swing high and low within a rolling window."""
    # Default Fibonacci levels if none are provided
    if fib_levels is None:
        fib_levels = [0.0, 0.236, 0.382, 0.5, 0.618, 1.0]

    # Identify the rolling window's recent swing high and low
    rolling_high = df['High'].rolling(window=lookback).max()
    rolling_low = df['Low'].rolling(window=lookback).min()

    # Calculate the difference between the swing high and low
    diff = rolling_high - rolling_low

    # Dictionary to store Fibonacci levels
    fib_dict = {}

    # Calculate Fibonacci levels dynamically
    for level in fib_levels:
        fib_dict[f'{level * 100}%'] = rolling_low + (level * diff)

    return fib_dict

def calculate_macd(df, fast=12, slow=26, signal=9):
    """Calculate MACD line and Signal line."""
    df['MACD_Line'] = df['Close'].ewm(span=fast, adjust=False).mean() - df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=signal, adjust=False).mean()
    return df

def main():
    # Define the stock symbol and time period
    symbol = 'APTV'
    start_date = '2014-12-19'
    end_date = '2024-12-19'

    # Download the data
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df.ffill(inplace=True)

    # Calculate Fibonacci levels for a rolling window of the last 252 days
    fib_levels = calculate_fibonacci_levels(df, lookback=126)

    # Calculate MACD
    df = calculate_macd(df, fast=24, slow=52, signal=18)

    # Define entry signal with Fibonacci and MACD confirmation
    df['Entry'] = (
        ((df['Close'] >= fib_levels['38.2%']) & (df['Close'] <= fib_levels['50.0%'])) &  # Price approaching key Fibonacci levels
        (df['MACD_Line'] > df['MACD_Signal'])  # MACD bullish crossover
    )

    # Define exit signal for a Buy Trade
    df['Exit'] = (
        (df['MACD_Line'] < df['MACD_Signal']) &  # MACD bearish crossover
        ((df['Close'] >= fib_levels['0.0%']) & (df['Close'] <= fib_levels['38.2%']))  # Price approaching key Fibonacci levels
    )

    # Convert signals to boolean arrays
    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Display performance metrics
    print(portfolio.stats())

    # Plot equity curve
    portfolio.plot().show()

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2014-12-19 00:00:00
End                           2024-12-18 00:00:00
Period                                       2516
Start Value                              100000.0
End Value                           180467.415655
Total Return [%]                        80.467416
Benchmark Return [%]                    -8.062235
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5432.244837
Max Drawdown [%]                         48.06844
Max Drawdown Duration                       845.0
Total Trades                                   17
Total Closed Trades                            17
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            23.529412
Best Trade [%]                         140.752088
Worst Trade [%]                        -12.996117
Avg Winning Trade [%]                   53.170019
Avg Losing Trade [%]                    -6.466048


In [37]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

def calculate_fibonacci_levels(df, lookback=50, fib_levels=None):
    """Calculate Fibonacci retracement levels based on recent swing high and low within a rolling window."""
    if fib_levels is None:
        fib_levels = [0.0, 0.236, 0.382, 0.5, 0.618, 1.0]

    rolling_high = df['High'].rolling(window=lookback).max()
    rolling_low = df['Low'].rolling(window=lookback).min()
    diff = rolling_high - rolling_low

    fib_dict = {}
    for level in fib_levels:
        fib_dict[f'{level * 100}%'] = rolling_low + (level * diff)

    return fib_dict

def calculate_macd(df, fast=12, slow=26, signal=9):
    """Calculate MACD line and Signal line."""
    df['MACD_Line'] = df['Close'].ewm(span=fast, adjust=False).mean() - df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=signal, adjust=False).mean()
    return df

def calculate_performance(symbol, start_date, end_date):
    """Calculate the performance of a single stock symbol."""
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df.ffill(inplace=True)

    # Calculate Fibonacci levels and MACD
    fib_levels = calculate_fibonacci_levels(df, lookback=126)
    df = calculate_macd(df, fast=12, slow=26, signal=9)

    # Define entry and exit signals
    df['Entry'] = (
        ((df['Close'] >= fib_levels['38.2%']) & (df['Close'] <= fib_levels['50.0%'])) &
        (df['MACD_Line'] > df['MACD_Signal'])
    )
    df['Exit'] = (
        (df['MACD_Line'] < df['MACD_Signal']) &
        ((df['Close'] >= fib_levels['0.0%']) & (df['Close'] <= fib_levels['38.2%']))
    )

    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Return portfolio performance (total return)
    return portfolio.total_return()

def calculate_buy_and_hold_performance(symbol, start_date, end_date):
    """Calculate the buy-and-hold performance of a single stock."""
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df.ffill(inplace=True)

    # Calculate total return for buy and hold strategy
    initial_price = df['Close'].iloc[0]
    final_price = df['Close'].iloc[-1]
    total_return = (final_price - initial_price) / initial_price

    return total_return

def main():
    # Define the S&P 500 stock symbols (this list can be fetched dynamically or updated manually)
    sp500_symbols = ['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMTM',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',
 'CPRT',
 'GLW',
 'CPAY',
 'CTVA',
 'CSGP',
 'COST',
 'CTRA',
 'CRWD',
 'CCI',
 'CSX',
 'CMI',
 'CVS',
 'DHR',
 'DRI',
 'DVA',
 'DAY',
 'DECK',
 'DE',
 'DELL',
 'DAL',
 'DVN',
 'DXCM',
 'FANG',
 'DLR',
 'DFS',
 'DG',
 'DLTR',
 'D',
 'DPZ',
 'DOV',
 'DOW',
 'DHI',
 'DTE',
 'DUK',
 'DD',
 'EMN',
 'ETN',
 'EBAY',
 'ECL',
 'EIX',
 'EW',
 'EA',
 'ELV',
 'EMR',
 'ENPH',
 'ETR',
 'EOG',
 'EPAM',
 'EQT',
 'EFX',
 'EQIX',
 'EQR',
 'ERIE',
 'ESS',
 'EL',
 'EG',
 'EVRG',
 'ES',
 'EXC',
 'EXPE',
 'EXPD',
 'EXR',
 'XOM',
 'FFIV',
 'FDS',
 'FICO',
 'FAST',
 'FRT',
 'FDX',
 'FIS',
 'FITB',
 'FSLR',
 'FE',
 'FI',
 'FMC',
 'F',
 'FTNT',
 'FTV',
 'FOXA',
 'FOX',
 'BEN',
 'FCX',
 'GRMN',
 'IT',
 'GE',
 'GEHC',
 'GEV',
 'GEN',
 'GNRC',
 'GD',
 'GIS',
 'GM',
 'GPC',
 'GILD',
 'GPN',
 'GL',
 'GDDY',
 'GS',
 'HAL',
 'HIG',
 'HAS',
 'HCA',
 'DOC',
 'HSIC',
 'HSY',
 'HES',
 'HPE',
 'HLT',
 'HOLX',
 'HD',
 'HON',
 'HRL',
 'HST',
 'HWM',
 'HPQ',
 'HUBB',
 'HUM',
 'HBAN',
 'HII',
 'IBM',
 'IEX',
 'IDXX',
 'ITW',
 'INCY',
 'IR',
 'PODD',
 'INTC',
 'ICE',
 'IFF',
 'IP',
 'IPG',
 'INTU',
 'ISRG',
 'IVZ',
 'INVH',
 'IQV',
 'IRM',
 'JBHT',
 'JBL',
 'JKHY',
 'J',
 'JNJ',
 'JCI',
 'JPM',
 'JNPR',
 'K',
 'KVUE',
 'KDP',
 'KEY',
 'KEYS',
 'KMB',
 'KIM',
 'KMI',
 'KKR',
 'KLAC',
 'KHC',
 'KR',
 'LHX',
 'LH',
 'LRCX',
 'LW',
 'LVS',
 'LDOS',
 'LEN',
 'LLY',
 'LIN',
 'LYV',
 'LKQ',
 'LMT',
 'L',
 'LOW',
 'LULU',
 'LYB',
 'MTB',
 'MPC',
 'MKTX',
 'MAR',
 'MMC',
 'MLM',
 'MAS',
 'MA',
 'MTCH',
 'MKC',
 'MCD',
 'MCK',
 'MDT',
 'MRK',
 'META',
 'MET',
 'MTD',
 'MGM',
 'MCHP',
 'MU',
 'MSFT',
 'MAA',
 'MRNA',
 'MHK',
 'MOH',
 'TAP',
 'MDLZ',
 'MPWR',
 'MNST',
 'MCO',
 'MS',
 'MOS',
 'MSI',
 'MSCI',
 'NDAQ',
 'NTAP',
 'NFLX',
 'NEM',
 'NWSA',
 'NWS',
 'NEE',
 'NKE',
 'NI',
 'NDSN',
 'NSC',
 'NTRS',
 'NOC',
 'NCLH',
 'NRG',
 'NUE',
 'NVDA',
 'NVR',
 'NXPI',
 'ORLY',
 'OXY',
 'ODFL',
 'OMC',
 'ON',
 'OKE',
 'ORCL',
 'OTIS',
 'PCAR',
 'PKG',
 'PLTR',
 'PANW',
 'PARA',
 'PH',
 'PAYX',
 'PAYC',
 'PYPL',
 'PNR',
 'PEP',
 'PFE',
 'PCG',
 'PM',
 'PSX',
 'PNW',
 'PNC',
 'POOL',
 'PPG',
 'PPL',
 'PFG',
 'PG',
 'PGR',
 'PLD',
 'PRU',
 'PEG',
 'PTC',
 'PSA',
 'PHM',
 'QRVO',
 'PWR',
 'QCOM',
 'DGX',
 'RL',
 'RJF',
 'RTX',
 'O',
 'REG',
 'REGN',
 'RF',
 'RSG',
 'RMD',
 'RVTY',
 'ROK',
 'ROL',
 'ROP',
 'ROST',
 'RCL',
 'SPGI',
 'CRM',
 'SBAC',
 'SLB',
 'STX',
 'SRE',
 'NOW',
 'SHW',
 'SPG',
 'SWKS',
 'SJM',
 'SW',
 'SNA',
 'SOLV',
 'SO',
 'LUV',
 'SWK',
 'SBUX',
 'STT',
 'STLD',
 'STE',
 'SYK',
 'SMCI',
 'SYF',
 'SNPS',
 'SYY',
 'TMUS',
 'TROW',
 'TTWO',
 'TPR',
 'TRGP',
 'TGT',
 'TEL',
 'TDY',
 'TFX',
 'TER',
 'TSLA',
 'TXN',
 'TPL',
 'TXT',
 'TMO',
 'TJX',
 'TSCO',
 'TT',
 'TDG',
 'TRV',
 'TRMB',
 'TFC',
 'TYL',
 'TSN',
 'USB',
 'UBER',
 'UDR',
 'ULTA',
 'UNP',
 'UAL',
 'UPS',
 'URI',
 'UNH',
 'UHS',
 'VLO',
 'VTR',
 'VLTO',
 'VRSN',
 'VRSK',
 'VZ',
 'VRTX',
 'VTRS',
 'VICI',
 'V',
 'VST',
 'VMC',
 'WRB',
 'GWW',
 'WAB',
 'WBA',
 'WMT',
 'DIS',
 'WBD',
 'WM',
 'WAT',
 'WEC',
 'WFC',
 'WELL',
 'WST',
 'WDC',
 'WY',
 'WMB',
 'WTW',
 'WYNN',
 'XEL',
 'XYL',
 'YUM',
 'ZBRA',
 'ZBH',
 'ZTS']  # Short list for example
    start_date = '2014-12-19'
    end_date = '2024-12-19'
    target_ratio = 10  # We want stocks that outperform the buy and hold return by 10x

    stock_performance = []

    # Loop through each stock in S&P 500 and calculate the return
    for symbol in sp500_symbols:
        print(f"Calculating performance for {symbol}...")

        # Calculate the buy-and-hold return
        buy_and_hold_return = calculate_buy_and_hold_performance(symbol, start_date, end_date)
        print(f"{symbol} buy-and-hold return: {buy_and_hold_return * 100:.2f}%")

        # Calculate the return of the trading strategy (with Fibonacci and MACD)
        stock_return = calculate_performance(symbol, start_date, end_date)
        print(f"{symbol} strategy return: {stock_return * 100:.2f}%")

        # Compare stock return to buy-and-hold return
        relative_return = stock_return / buy_and_hold_return
        stock_performance.append((symbol, relative_return))

    # Sort the stock performance based on relative return (highest to lowest)
    stock_performance.sort(key=lambda x: x[1], reverse=True)

    # Print the rank from best to worst
    print("\nRanking of stocks by performance (relative to buy and hold):")
    for rank, (symbol, relative_return) in enumerate(stock_performance, start=1):
        print(f"Rank {rank}: {symbol} - Outperforms buy and hold by {relative_return:.2f} times")

if __name__ == "__main__":
    main()


Calculating performance for MMM...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MMM buy-and-hold return: -9.27%
MMM strategy return: 17.35%
Calculating performance for AOS...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AOS buy-and-hold return: 153.07%
AOS strategy return: 52.27%
Calculating performance for ABT...


...

ZBRA buy-and-hold return: 391.00%
ZBRA strategy return: 225.27%
Calculating performance for ZBH...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ZBH buy-and-hold return: -5.11%
ZBH strategy return: -6.87%
Calculating performance for ZTS...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ZTS buy-and-hold return: 283.84%
ZTS strategy return: 273.23%

Ranking of stocks by performance (relative to buy and hold):
Rank 1: DLTR - Outperforms buy and hold by 43.09 times
Rank 2: EQT - Outperforms buy and hold by 20.92 times
Rank 3: WMB - Outperforms buy and hold by 7.27 times
Rank 4: GE - Outperforms buy and hold by 7.09 times
Rank 5: CPB - Outperforms buy and hold by 6.84 times
Rank 6: ARE - Outperforms buy and hold by 6.82 times
Rank 7: LVS - Outperforms buy and hold by 6.58 times
Rank 8: HAS - Outperforms buy and hold by 5.79 times
Rank 9: KIM - Outperforms buy and hold by 5.46 times
Rank 10: OKE - Outperforms buy and hold by 4.46 times
Rank 11: FIS - Outperforms buy and hold by 4.12 times
Rank 12: DG - Outperforms buy and hold by 4.09 times
Rank 13: MDT - Outperforms buy and hold by 3.96 times
Rank 14: LUV - Outperforms buy and hold by 3.95 times
Rank 15: INCY - Outperforms buy and hold by 3.80 times
Rank 16: CAG - Outperforms buy and hold by 3.77 times
Rank 17: APTV - Out

In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

def calculate_fibonacci_levels(df, lookback=50, fib_levels=None):
    """Calculate Fibonacci retracement levels based on recent swing high and low within a rolling window."""
    # Default Fibonacci levels if none are provided
    if fib_levels is None:
        fib_levels = [0.0, 0.236, 0.382, 0.5, 0.618, 1.0]

    # Identify the rolling window's recent swing high and low
    rolling_high = df['High'].rolling(window=lookback).max()
    rolling_low = df['Low'].rolling(window=lookback).min()

    # Calculate the difference between the swing high and low
    diff = rolling_high - rolling_low

    # Dictionary to store Fibonacci levels
    fib_dict = {}

    # Calculate Fibonacci levels dynamically
    for level in fib_levels:
        fib_dict[f'{level * 100}%'] = rolling_low + (level * diff)

    return fib_dict

def calculate_macd(df, fast=12, slow=26, signal=9):
    """Calculate MACD line and Signal line."""
    df['MACD_Line'] = df['Close'].ewm(span=fast, adjust=False).mean() - df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=signal, adjust=False).mean()
    return df

def main():
    # Define the stock symbol and time period
    symbol = 'TSLA'
    start_date = '2014-12-17'
    end_date = '2024-12-17'

    # Download the data
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df.ffill(inplace=True)

    # Calculate Fibonacci levels for a rolling window of the last 252 days
    fib_levels = calculate_fibonacci_levels(df, lookback=126)

    # Calculate MACD
    df = calculate_macd(df, fast=12, slow=26, signal=9)

    # Define entry signal with Fibonacci and MACD confirmation
    df['Entry'] = (
        ((df['Close'] >= fib_levels['38.2%']) & (df['Close'] <= fib_levels['50.0%'])) &  # Price approaching key Fibonacci levels
        (df['MACD_Line'] > df['MACD_Signal'])  # MACD bullish crossover
    )

    # Define exit signal for a Buy Trade
    df['Exit'] = (
        (df['MACD_Line'] < df['MACD_Signal']) &  # MACD bearish crossover
        ((df['Close'] >= fib_levels['0.0%']) & (df['Close'] <= fib_levels['38.2%']))  # Price approaching key Fibonacci levels
    )

    # Convert signals to boolean arrays
    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Display performance metrics
    print(portfolio.stats())

    # Plot equity curve
    portfolio.plot().show()

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2014-12-17 00:00:00
End                           2024-12-16 00:00:00
Period                                       2516
Start Value                              100000.0
End Value                          1898816.012355
Total Return [%]                      1798.816012
Benchmark Return [%]                  3274.453518
Max Gross Exposure [%]                      100.0
Total Fees Paid                      26541.301515
Max Drawdown [%]                        63.326967
Max Drawdown Duration                       777.0
Total Trades                                   28
Total Closed Trades                            27
Total Open Trades                               1
Open Trade PnL                     1129367.607269
Win Rate [%]                            25.925926
Best Trade [%]                         455.073708
Worst Trade [%]                        -12.777505
Avg Winning Trade [%]                   93.765448
Avg Losing Trade [%]                    -5.934256
